In [2]:
from time import time

import numpy as np
import pyopencl as cl

In [5]:
%load_ext pyopencl.ipython_ext

The pyopencl.ipython_ext extension is already loaded. To reload it, use:
  %reload_ext pyopencl.ipython_ext


In [6]:
ctx = cl.create_some_context(interactive=True)
queue = cl.CommandQueue(ctx, properties=cl.command_queue_properties.PROFILING_ENABLE)

Choose platform:
[0] <pyopencl.Platform 'Apple' at 0x7fff0000>
Choice [0]:0
Choose device(s):
[0] <pyopencl.Device 'Intel(R) Core(TM) i7-4870HQ CPU @ 2.50GHz' on 'Apple' at 0xffffffff>
[1] <pyopencl.Device 'Iris Pro' on 'Apple' at 0x1024500>
[2] <pyopencl.Device 'AMD Radeon R9 M370X Compute Engine' on 'Apple' at 0x1021c00>
Choice, comma-separated [0]:2
Set the environment variable PYOPENCL_CTX='0:2' to avoid being asked again.


In [7]:
for device in ctx.get_info(cl.context_info.DEVICES):
  print("Device name:", device.name)
  print("Device type:", cl.device_type.to_string(device.type))
  print("Device memory: ", device.global_mem_size//1024//1024, 'MB')
  print("Device max clock speed:", device.max_clock_frequency, 'MHz')
  print("Device compute units:", device.max_compute_units)

Device name: AMD Radeon R9 M370X Compute Engine
Device type: GPU
Device memory:  2048 MB
Device max clock speed: 80 MHz
Device compute units: 10


In [8]:
%%cl_kernel -o "-cl-fast-relaxed-math"

__kernel
void mmul(__global const float* a, __global const float* b, __global float* c){
    int i = get_global_id(0); 
    c[i] = 0;
    for(int j = 0; j < 300000; j++)
    {
        c[i] += a[i * 100 + j] * b[j]; 
    }
}

In [10]:
%%cl_kernel -o "-cl-fast-relaxed-math"

__kernel
void mdot(__global const float* a, 
          __global const float* b, 
          __global float* c, 
          const int n, const int p, 
          __global int* offsets, 
          __local float* temp)
{
    int glid = get_global_id(0); 
    int pos = glid;
    int offset = offsets[glid];
    for(int i = 0; i < n; i++)
    {
        int idx = offset + a[pos]; 
        temp[idx] += b[i];
        pos += p;
    }
    for(int i = 0; i < n; i++){
        c[i] = temp[i];
    }
}

In [158]:
%%cl_kernel -o "-cl-fast-relaxed-math"

__kernel
void mdot(__global const uint8* a, __global const float* b, __global float* c, const int n){
    int i = get_global_id(0); 
    float d = 0;
    for(int j = 0; j < n; j++)
    {
        d += a[j * 100 + i] * b[j]; 
    }
    c[i] = d;
    
}

In [159]:
%%cl_kernel -o "-cl-fast-relaxed-math"

__kernel 
void dot_product(__global float4* a_vec, 
                 __global float4* b_vec,
                 __global float* output, 
                 __local float4* partial_dot) {

   int gid = get_global_id(0);
   int lid = get_local_id(0);
   int group_size = get_local_size(0);

   partial_dot[lid] = a_vec[gid] * b_vec[gid];
   barrier(CLK_LOCAL_MEM_FENCE);

   for(int i = group_size/2; i>0; i >>= 1) {
      if(lid < i) {
         partial_dot[lid] += partial_dot[lid + i];
      }
      barrier(CLK_LOCAL_MEM_FENCE);
   }

   if(lid == 0) {
      output[get_group_id(0)] = dot(partial_dot[0], (float4)(1.0f));
   }
}

In [160]:
n = 300000
p = 1000
A = np.random.rand(n * p).astype(np.float32).reshape((n, p))
B = np.random.rand(n).astype(np.float32)
C = np.zeros(p).astype(np.float32)

In [161]:
A_buf = cl.Buffer(ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=A)
B_buf = cl.Buffer(ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=B)
C_buf = cl.Buffer(ctx, cl.mem_flags.WRITE_ONLY, C.nbytes)

In [162]:
#mvmul.set_args(A_buf, B_buf, C_buf)
mdot.set_args(A_buf, B_buf, C_buf, np.int32(n))

In [163]:
cl.get_platforms()[0].get_devices(cl.device_type.ALL)[2].max_work_group_size #max_work_item_dimensions

256

In [164]:
ev = cl.enqueue_nd_range_kernel(queue, mdot, (p,), None)
ev.wait()
gpu_start_time = time()  # Get the GPU start time
ev = cl.enqueue_nd_range_kernel(queue, mdot, (p,), None)
cl.enqueue_read_buffer(queue, C_buf, C)
queue.finish()
gpu_end_time = time()  # Get the GPU end time
elapsed = 1e-9 * (ev.profile.end - ev.profile.start)  # Calculate the time it took to execute the kernel
print("GPU Kernel Time: {0} s".format(elapsed))  # Print the time it took to execute the kernel
print("GPU Time: {0} s".format(gpu_end_time - gpu_start_time))  # Print the time the GPU program took, including 
                                                                #both memory copies
print((n * p)/(gpu_end_time - gpu_start_time) / (1024 * 1024 * 1024), "GFlops")

GPU Kernel Time: 0.11591096000000001 s
GPU Time: 0.1342761516571045 s
2.080762435746056 GFlops


/Users/Axa/git/pricing-tool/venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: 'enqueue_read_buffer' has been deprecated in version 2011.1. Please use enqueue_copy() instead.
  """


In [165]:
C

array([75008.414, 74864.51 , 75034.93 , 74927.42 , 75070.87 , 75055.29 ,
       74818.57 , 75180.5  , 74918.96 , 74954.12 , 75095.9  , 74875.67 ,
       74979.26 , 74972.89 , 74906.58 , 74910.5  , 75006.17 , 74995.   ,
       74920.945, 74995.016, 74909.08 , 74821.73 , 74991.53 , 75054.28 ,
       74977.77 , 74901.734, 74850.49 , 74906.57 , 74944.38 , 74984.195,
       75021.71 , 74982.53 , 75006.44 , 74936.08 , 74938.66 , 74961.18 ,
       74929.75 , 74911.71 , 74920.16 , 75039.01 , 75101.414, 75007.49 ,
       74991.62 , 74937.16 , 74956.93 , 74985.68 , 75020.9  , 74995.336,
       75040.266, 74997.875, 74966.18 , 74994.22 , 74981.06 , 74998.98 ,
       75088.695, 74902.76 , 74941.055, 74907.25 , 75145.484, 74923.29 ,
       75079.555, 74975.13 , 75222.99 , 74995.336, 75028.805, 75004.5  ,
       75170.4  , 75022.57 , 75097.266, 75164.19 , 74823.96 , 74938.27 ,
       75104.27 , 74913.49 , 75021.91 , 74955.22 , 74937.56 , 74834.734,
       75166.38 , 74942.695, 74881.17 , 75071.484, 

In [166]:
%%timeit
(B[:, np.newaxis] * A).sum(axis=0)

1 s ± 59.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
cpu_start_time = time()  # Get the GPU start time
D = np.dot(A, B)
cpu_end_time = time()  # Get the GPU end time
print("CPU Time: {0} s".format(cpu_end_time - cpu_start_time))  
print((n * p * 2)/(cpu_end_time - cpu_start_time) / (1024 * 1024 * 1024), "GFlops")

CPU Time: 0.008141040802001953 s
6.8639079247935335 GFlops


In [31]:
C

array([24.567423, 22.538635, 23.54259 , ..., 25.078547, 25.717302,
       26.456945], dtype=float32)

In [32]:
np.dot(A, B)

array([24.567429, 22.538643, 23.542585, ..., 25.07854 , 25.717293,
       26.456944], dtype=float32)

In [33]:
(cpu_end_time - cpu_start_time) / (gpu_end_time - gpu_start_time)

0.23217041877163042

In [301]:
C

array([28.689331, 26.380375, 26.65357 , ..., 23.372551, 26.52175 ,
       27.61783 ], dtype=float32)

In [302]:
D

array([28.689327, 26.380373, 26.653566, ..., 23.372551, 26.52174 ,
       27.617823], dtype=float32)

In [303]:
import pyclblast

from pyopencl.array import Array

In [304]:
# Settings for this sample
dtype = 'float32'
alpha = 1
beta = 0

print("# Setting up Numpy arrays")
y = np.random.rand(n).astype(dtype=dtype)

print("# Setting up OpenCL arrays")
cla = Array(queue, A.shape, A.dtype)
clx = Array(queue, B.shape, B.dtype)
cly = Array(queue, y.shape, y.dtype)
cla.set(A)
clx.set(x)
cly.set(y)

# Setting up Numpy arrays
# Setting up OpenCL arrays


In [305]:
pyclblast.gemv(queue, n, p, cla, clx, cly, a_ld=p, alpha=alpha, beta=beta)
queue.finish()
print("# Result for vector y: %s" % cly.get())
#print("# Expected result:     %s" % (alpha * np.dot(A, B) + beta * y))

# Result for vector y: [27.239933 25.933067 31.403833 ... 24.658667 28.381613 26.393766]


In [306]:
(alpha * np.dot(A, B) + beta * y)

array([28.689327, 26.380373, 26.653566, ..., 23.372551, 26.52174 ,
       27.617823], dtype=float32)